In [2]:
import pickle
import sys
from pathlib import Path
import json

import numpy as np


from towerPrimitives import primitives
from makeTowerTasks import *
from grammar import *
from fragmentGrammar import *
from gen_seq import *
import utilities
from enumeration import *
from program import *
import render
import importlib


_ = importlib.reload(utilities)

(2x1 (left 4 (#(lambda (2x1 (left 1 (1x2 (1x2 $0))))) $0)))


## Setup

In [6]:
#config

ws = [1.5, 3.2, 3.3, 9.6] #subset of ws used when running model on all trial sequences
w_locations = {w: i for i, w in enumerate(ws)}

# ground-truth programs fed into library learning
tower_strings = {"CL" :"(h (l 1) v v (r 1) h (r 12) h (l 4) h (l 1) v v)",
                "CPi": "(h (l 1) v v (r 1) h (r 6) v (r 6) v (l 5) h (r 4) h)",
                "LPi": "(h (l 4) h (l 1) v v (r 9) v (r 6) v (l 5) h (r 4) h)",
                "LC": "(h (l 4) h (l 1) v v (r 12) h (l 1) v v (r 1) h)",
                "PiC": "(v (r 6) v (l 5) h (r 4) h (r 7) h (l 1) v v (r 1) h)",
                "PiL": "(v (r 6) v (l 5) h (r 4) h (r 9) h (l 4) h (l 1) v v)"}

# webppl-readable ground-truth programs
manual_tower_programs = {"CL" :"h l_1 v v r_1 h r_12 h l_4 h l_1 v v",
                         "CPi": "h l_1 v v r_1 h r_6 v r_6 v l_5 h r_4 h",
                         "PiC": "v r_6 v l_5 h r_4 h r_7 h l_1 v v r_1 h",
                         "LPi": "h l_4 h l_1 v v r_9 v r_6 v l_5 h r_4 h",
                         "LC": "h l_4 h l_1 v v r_12 h l_1 v v r_1 h",
                         "PiL": "v r_6 v l_5 h r_4 h r_9 h l_4 h l_1 v v"}


# base dsl for webppl-readable strings
base_dsl = ['h', 
 'v', 
 'l_0',
 'l_1',
 'l_2',
 'l_3',
 'l_4',
 'l_5',
 'l_6',
 'l_7',
 'l_8',
 'l_9',
 'l_10',
 'l_11',
 'l_12',
 'r_0',
 'r_1',
 'r_2',
 'r_3',
 'r_4',
 'r_5',
 'r_6',
 'r_7',
 'r_8',
 'r_9',
 'r_10',
 'r_11',
 'r_12']

In [7]:
def parse(s, base_dsl_only = False):
    '''
    Converts a program in lambda format (i.e. from Dreamcoder enumeration) into a sequence of commands, 
        possibly including learned chunks.
        
    base_dsl_only: output program in terms of base dsl commands only, 
                    rather than printing names of learned chunks e.g. 'chunk_8'

    # demo string without chunks to check hs and vs
    >>> s = '(lambda ((lambda (2x1 (left 4 ((lambda (2x1 (left 1 (1x2 (1x2 $0))))) $0)))) (right 9 (#(lambda (1x2 (right 6 (1x2 (left 5 (2x1 (right 4 (2x1 $0)))))))) (left 9 $0)))))'
    >>> parse(s) 
    'h l_4 h l_1 v v r_9 chunk_Pi l_9'
    
    '''
    
    s = utilities.parseSExpression(s)
#     print(s)
    def p(e):
#         print(e)
        if isinstance(e,list):
            if e[0] == '#':
                assert len(e) == 2
                if base_dsl_only:
                    return(p(e[1]))
                else:
                    return 'chunk_' + render.lookup[str(utilities.unparseSExpression(e[1]))][0].name + ' '
#                 return 'search_for_chunk ' # in dsl- lookup and return
            if e[0] == 'lambda':
                assert len(e) == 2
                return p(e[1]) # dig to see what else is in lambda 
            if e[0] == 'left':
                if (e[1] == '$1') or (e[1] == '$0'):
                    #return 'l_' + (p(e[2:]))
                    return ''
                else:
                    return 'l_' + e[1] + ' ' + (p(e[2:]))
            if e[0] == 'right': 
                if (e[1] == '$1') or (e[1] == '$0'):
                    #return 'r_' + (p(e[2:]))
                    return ''
                else:
                    return 'r_' + e[1] + ' ' + (p(e[2:]))
            if e[0] == '1x2': return 'v ' + (p(e[1:]))
            if e[0] == '2x1': return 'h ' + (p(e[1:]))
            f = ''
            for x in e:
                f = f + p(x)
            return f
        assert isinstance(e,str)
        if e[0] == '1x2': return 'v ' + (p(e[1:]))
        if e[0] == '2x1': return 'h ' + (p(e[1:]))
        if e == '$0': return ''
        if e == '$1': return ''
        raise ParseFailure((s,e))
    return p(s)[:-1]


def get_partially_chunked_programs(trial_datum):
    chunk_lambdas = trial_datum['dsl_lambda'][16:]
    chunk_names = trial_datum['chunks']
    
#     print('whole_lambdas', trial_datum['dsl_lambda'])
#     print('lambdas', chunk_lambdas)
#     print('names', chunk_names)
    
    progs = [trial_datum['min_program']]

    for prog in progs:
        for i, chunk_name in enumerate(chunk_names):
            new_prog = prog.replace(chunk_name, parse(chunk_lambdas[i], base_dsl_only=True))
            if not(new_prog in progs):
                progs.append(new_prog)
    
    progs_with_length = {p: len(p.split(' ')) for p in progs}
    return progs_with_length


## Load DSLs for each trial sequence

We are considering how abstractions are learned by individual participants across a sequence of trials.  
We therefore track how each participant's Domain Specific Language (*DSL*) changes across the trial sequence.  
DSLs include both the *base DSL* (specified above) as well as *program fragments* (abstractions) learned by Dreamcoder.  
Dreamcoder is run on the sequence of tower programs seen up to the current trial.  

`dsls[i]` is the dsl learned up to *and including* trial i.  
Use `dsls[i-1]` for learning (as we are considering how the DSL, up to this point, is used to refactor the current program).

In [3]:
data_path = './results/revised/'

dsls = {}
trial_seqs = {}

for ppt in range(1,50):
    
    dsls[ppt] = {}
    
    with open(data_path+f"{ppt}/configs.p", "rb") as config_file:
            trial_seqs[ppt] = pickle.load(config_file)
    
    for trial in range(1, 13):
        with open(data_path+f"{ppt}/{trial}.p", "rb") as input_file:
                dsls[ppt][trial] = pickle.load(input_file)

## Enumerate programs

We have the library of abstractions for each trial. Now we need to infer the actual programs used for the tower at that trial.  
We assume that participants can find the shortest possible program using their library of abstractions.  
We find these programs by:
1. Enumerating programs until they meet the specification. (This works for programs with large abstractions, but timesout for those with a small DSL.)
2. Manually refactoring programs written in the base DSL by searching for and replacing substrings with their corresponding chunk. (This is possible because in practice only a small handful of abstractions are learnt.)

In [14]:
# get the trial grammars for a single participant's trial sequence

# for a single value of w
w = 3.3
w_position = w_locations[w]

trial_tasks = [SupervisedTower(tower_pair + str(i+1), tower_strings[tower_pair]) for i, tower_pair in enumerate(trial_seqs[ppt])]

# get grammar for each trial
trial_grammars = {trial_tasks[i]: Grammar.uniform(primitives + dsls[ppt][i][w_position]) for i in range (1,12)}
trial_grammars[trial_tasks[0]] = Grammar.uniform(primitives)

### 1. Run program enumeration (need to rerun)

In [21]:
run = True

if run:
    for ppt in range(1,50):

        # for a single value of w
        w = 3.3
        w_position = w_locations[w]


        trial_tasks = [SupervisedTower(tower_pair + str(i+1), tower_strings[tower_pair]) for i, tower_pair in enumerate(trial_seqs[ppt])]

        # get grammar for each trial
        trial_grammars = {trial_tasks[i]: Grammar.uniform(primitives + dsls[ppt][i][w_position]) for i in range (1,12)}
        trial_grammars[trial_tasks[0]] = Grammar.uniform(primitives)

        filepath = './results/enumeration_ms/ppt_'+str(ppt)+'/'
        Path(filepath).mkdir(parents=True, exist_ok=True)

        f_test = multicoreEnumeration(trial_grammars, 
                                 trial_tasks, 
                                 maximumFrontier=10.0, 
                                 enumerationTimeout=20, 
                                 solver='python',
                                 filepath=filepath,
                                 filename='enumeration_50_ppt'+str(ppt))

        pickle.dump(f_test, open( './results/enumeration_ms/ppt_'+str(ppt)+'_complete.p', "wb" ))


(python) Launching tower -> tower (3 tasks) w/ 1 CPUs. 0.000000 <= MDL < 1.500000. Timeout 20.000000.
Process Process-1:
Traceback (most recent call last):
  File "/Users/will/opt/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/will/opt/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/will/compositional-abstractions/model_ms/lib_learning/utilities.py", line 338, in _launchParallelProcess
    [f, a, k] = PARALLELPROCESSDATA
TypeError: cannot unpack non-iterable NoneType object


#### load programs

In [10]:
# load programs from files

frontiers = {}
enumerated_programs = {}

for ppt in range(1,50):

    # for a single value of w
    w = 3.3
    w_position = w_locations[w]
    
    enumerated_programs[ppt] = {}

    filepath = './results/enumeration_cogsci_revised/ppt_'+str(ppt)+'_complete.p'
    
    with open(filepath, "rb") as input_file:
         frontiers[ppt] = pickle.load(input_file)
            
    
    for trial_num in range(1,13):
        enumerated_programs[ppt][trial_num] = [e.program for e in frontiers[ppt][0][trial_num-1].entries]

In [13]:
# print programs from enumeration

ppt_num = 1

[list(map(lambda x: parse(str(x)), 
          enumerated_programs[ppt_num][trial_num])) 
    for trial_num in range(1,13)]

[[],
 [],
 [],
 [],
 [],
 ['chunk_Pi r_9 chunk_L'],
 ['chunk_L r_12 chunk_C'],
 ['chunk_C r_6 chunk_Pi'],
 ['chunk_L r_9 chunk_Pi'],
 ['chunk_C r_12 chunk_L'],
 ['chunk_Pi r_7 chunk_C'],
 ['chunk_Pi r_9 chunk_L']]

### Refactor missing programs by substituting in learned abstractions

Programs involving fewers (or smaller) chunks are harder to reach through enumeration.
Here we find the shortest programs analytically.
- For the trials where no chunks have been learned, we use the ground-truth programs that were fed into Dreamcoder.
- For trials where chunks were learned, we find start with the ground-truth programs and swap in chunks by searching for equivalent substrings.

In [53]:
# Loop 

verbose = False

for ppt in range(1,50):
    
    trial_data = []

    # towerpairs plus trial numbers
    trial_tasks = [SupervisedTower(tower_pair + str(i+1), tower_strings[tower_pair]) for i, tower_pair in enumerate(trial_seqs[ppt])]

    # get dsls for all trials
    trial_grammars = {trial_tasks[i]: Grammar.uniform(primitives + dsls[ppt][i][w_position]) for i in range (1,12)}
    trial_grammars[trial_tasks[0]] = Grammar.uniform(primitives)
    
    for trial_num in range(1,13):
        
        scene = trial_seqs[ppt][trial_num-1] # get trial scene
        chunks = list(map(lambda x: parse(str(x)), trial_grammars[trial_tasks[trial_num-1]].primitives[16:]))
        manual_program = manual_tower_programs[scene]
        min_program = manual_program

        if (len(enumerated_programs[ppt][trial_num]) > 0): # see if program found in enumeration

            # parse program
            parsed_programs = list(map(lambda x: parse(str(x)), enumerated_programs[ppt][trial_num]))

            # trial_programs = []
            # for entry in g[0][trial_num-1].entries:
            #     trial_programs.append(parse(str(entry.program), base_dsl_only=False))

            # Find shortest program if multiple found (in practice, only one program is found)
            ps = {p: len(p.split(' ')) for p in parsed_programs}
            trial_programs_sorted = {k: v for k, v in sorted(ps.items(), key=lambda item: item[1])}

            min_program = list(trial_programs_sorted.keys())[0] # get first, shortest program

            if verbose: print(min_program)

        elif len(dsls[ppt][trial_num][w_position]) == 0: # if no chunks learned, take input program
            
            if verbose: print(min_program)

        else: # if some chunks learned, swap in chunks into input program


            # with trailing rights or lefts trimmed off
            # WARNING- THIS ALLOWS CHUNKS WITH TRAILING RIGHT TO BE USED IN PLACES WHERE THERE ISN'T A RIGHT MOVE AT THE END
            chunk_tranlations_trimmed = list(map(lambda x: parse(str(x), base_dsl_only=True), dsls[ppt][trial_num][w_position]))

            chunked_program = manual_program

            # WARNING: this goes through chunks in order. Different orders might yield different programs
            # We should probably try all programs, then find the minimum
            
            for i, chunk in enumerate(chunks):
                if chunk in ['chunk_Pi','chunk_L','chunk_C']:
                    chunked_program = chunked_program.replace(chunk_tranlations_trimmed[i], chunk)
                    
            for i, chunk in enumerate(chunks):
                if chunk in ['chunk_8','chunk_8b']:
                    chunked_program = chunked_program.replace(chunk_tranlations_trimmed[i], chunk)

            if verbose: print(chunked_program)
            
            min_program = chunked_program

            
#         ps = {p: len(p.split(' ')) for p in parsed_programs}
#         trial_programs_sorted = {k: v for k, v in sorted(ps.items(), key=lambda item: item[1])}

        trial_data.append(
            {
            'ppt' : ppt, # just added
            'trial_num': trial_num,
            'towers': scene,
            'dsl_lambda': [str(p) for p in trial_grammars[trial_tasks[trial_num-1]].primitives],
            'chunks': chunks,
            'dsl': base_dsl + list(map(lambda s: parse(str(s)), trial_grammars[trial_tasks[trial_num-1]].primitives[16:])),
            #         'trial_programs': trial_programs_sorted,
            'min_program': min_program
            })
    
    # find programs with abstractions replaced with base dsl only
    for i, trial_datum in enumerate(trial_data):
        trial_datum['programs_with_length'] = get_partially_chunked_programs(trial_datum)

    # # This will save results within lib-learning directory. 
    with open("../lib_learning_output/synthesis_ms_output/ca_synthesis_ms_ppt_" + str(ppt) + ".json", "w") as write_file:
         json.dump(trial_data, write_file)


Now programs are stored in `lib_learning_output/synthesis_ms_output`

In [79]:
# inspect programs

# cogsci programs look wrong! Looks like they're all the same for the last 6 trials!

ppt = 4

with open("../lib_learning_output/synthesis_output_cogsci_revised/ca_synthesis_cogsci_21_ppt_" + str(ppt) + ".json", "r") as read_file:
    cogsci_programs = json.load(read_file)

with open("../lib_learning_output/synthesis_ms_output/ca_synthesis_ms_ppt_" + str(ppt) + ".json", "r") as read_file:
    ms_programs = json.load(read_file)


ms_progs = [trial_datum['min_program'] for trial_datum in ms_programs]
cs_progs = [trial_datum['min_program'] for trial_datum in cogsci_programs]

ms_progs
# cs_progs

# ms_progs == cs_progs

['h l_1 v v r_1 h r_6 v r_6 v l_5 h r_4 h',
 'h l_4 h l_1 v v r_12 h l_1 v v r_1 h',
 'h l_4 h l_1 v v r_9 v r_6 v l_5 h r_4 h',
 'chunk_8b r_1 h r_12 h l_4 chunk_8b',
 'v r_6 v l_5 h r_4 h r_9 h l_4 chunk_8',
 'chunk_Pi r_7 chunk_C',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L']

In [81]:
# show results for one participant
ppt_programs = [trial_datum['min_program'] for trial_datum in ms_programs]
ppt_programs

['h l_1 v v r_1 h r_6 v r_6 v l_5 h r_4 h',
 'h l_4 h l_1 v v r_12 h l_1 v v r_1 h',
 'h l_4 h l_1 v v r_9 v r_6 v l_5 h r_4 h',
 'chunk_8b r_1 h r_12 h l_4 chunk_8b',
 'v r_6 v l_5 h r_4 h r_9 h l_4 chunk_8',
 'chunk_Pi r_7 chunk_C',
 'chunk_L r_9 chunk_Pi',
 'chunk_C r_6 chunk_Pi',
 'chunk_L r_12 chunk_C',
 'chunk_Pi r_7 chunk_C',
 'chunk_C r_12 chunk_L',
 'chunk_Pi r_9 chunk_L']

In [84]:
ppt_programs = [trial_datum['min_program'] for trial_datum in cogsci_programs]
ppt_programs

['h l_4 h l_1 v v r_9 v r_6 v l_5 h r_4 h',
 'h l_1 v v r_1 h r_6 v r_6 v l_5 h r_4 h',
 'h l_4 h l_1 v v r_12 h l_1 v v r_1 h',
 'v r_6 v l_5 h r_4 h r_9 h l_4 chunk_8b',
 'chunk_8 r_7 chunk_Pi r_1 h',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L',
 'chunk_Pi r_9 chunk_L']

In [ ]:
# used for cogsci (I think)- doing something wrong (using variable that wasn't getting updated)
# Create trial data by looping through participants and trials

verbose = False

for ppt in range(1,50):
    
    trial_data = []

    # towerpairs plus trial numbers
    trial_tasks = [SupervisedTower(tower_pair + str(i+1), tower_strings[tower_pair]) for i, tower_pair in enumerate(trial_seqs[ppt])]

    # get dsls for all trials
    trial_grammars = {trial_tasks[i]: Grammar.uniform(primitives + dsls[ppt][i][w_position]) for i in range (1,12)}
    trial_grammars[trial_tasks[0]] = Grammar.uniform(primitives)
    
    for trial_num in range(1,13):
        
        scene = trial_seqs[ppt][trial_num-1] # get trial scene
        chunks = list(map(lambda x: parse(str(x)), trial_grammars[trial_tasks[trial_num-1]].primitives[16:]))
        manual_program = manual_tower_programs[scene]
        min_program = manual_program

        if (len(enumerated_programs[ppt][trial_num]) > 0): # see if program found in enumeration

            # parse program
            parsed_programs = list(map(lambda x: parse_2(str(x)), enumerated_programs[ppt][trial_num]))

            min_program = list(trial_programs_sorted.keys())[0] # get first, shortest program

            if verbose: print(min_program)

        elif len(dsls[ppt][trial_num][w_position]) == 0: # if no chunks learned, take input program
            
            if verbose: print(min_program)

        else: # if some chunks learned, swap in chunks into input program


            # with trailing rights or lefts trimmed off
            # WARNING- THIS ALLOWS CHUNKS WITH TRAILING RIGHT TO BE USED IN PLACES WHERE THERE ISN'T A RIGHT MOVE AT THE END
            chunk_tranlations_trimmed = list(map(lambda x: parse_2(str(x), base_dsl_only=True), dsls[ppt][trial_num][w_position]))

            chunked_program = manual_program

            # WARNING: this goes through chunks in order. Different orders might yield different programs
            # We should probably try all programs, then find the minimum
            
            for i, chunk in enumerate(chunks):
                if chunk in ['chunk_Pi','chunk_L','chunk_C']:
                    chunked_program = chunked_program.replace(chunk_tranlations_trimmed[i], chunk)
                    
            for i, chunk in enumerate(chunks):
                if chunk in ['chunk_8','chunk_8b']:
                    chunked_program = chunked_program.replace(chunk_tranlations_trimmed[i], chunk)

            if verbose: print(chunked_program)
            
            min_program = chunked_program

            
#         ps = {p: len(p.split(' ')) for p in parsed_programs}
#         trial_programs_sorted = {k: v for k, v in sorted(ps.items(), key=lambda item: item[1])}

        trial_data.append(
            {
            'ppt' : ppt, # just added
            'trial_num': trial_num,
            'towers': scene,
            'dsl_lambda': [str(p) for p in trial_grammars[trial_tasks[trial_num-1]].primitives],
            'chunks': chunks,
            'dsl': base_dsl + list(map(lambda s: parse(str(s)), trial_grammars[trial_tasks[trial_num-1]].primitives[16:])),
            #         'trial_programs': trial_programs_sorted,
            'min_program': min_program
            })
    
    # find programs with abstractions replaced with base dsl only
    for i, trial_datum in enumerate(trial_data):
        trial_datum['programs_with_length'] = get_partially_chunked_programs(trial_datum)

    # This will save results within lib-learning directory. 
#     with open("results/revised/synthesis_output/ca_synthesis_cogsci_21_ppt_" + str(ppt) + ".json", "w") as write_file:
#          json.dump(trial_data, write_file)


NameError: name 'parse_2' is not defined

### Used in original cogsci submission

In [190]:
# manually add trials 1-5

trial_data = []

for trial_num in range(1,6):
    
#     trial_programs = []
#     for entry in g[0][trial_num-1].entries:
#         trial_programs.append(parse(str(entry.program), base_dsl_only=False))

    manual_program = manual_tower_programs[trial_seq[trial_num-1]]

    ps = {manual_program: len(manual_program.split(' '))}
    trial_programs_sorted = {k: v for k, v in sorted(ps.items(), key=lambda item: item[1])}
    
    trial_data.append(
    {
        'trial_num': trial_num,
        'towers': trial_seq[trial_num-1],
        'dsl_lambda': [str(p) for p in trial_grammars[trial_tasks[trial_num-1]].primitives],
        'chunks': list(map(lambda s: parse(str(s)), trial_grammars[trial_tasks[trial_num-1]].primitives[16:])),
        'dsl': base_dsl + list(map(lambda s: parse(str(s)), trial_grammars[trial_tasks[trial_num-1]].primitives[16:])),
#         'trial_programs': trial_programs_sorted,
        'min_program': manual_program
    })
    

In [191]:
# manually add min-length programs with chunks from trials four and five.

# trial 4
p_3 = 'chunk_8 r_1 h r_12 h l_4 h l_1 v v'
trial_data[3]['min_program'] = p_3

# trial 5
p_4 = 'chunk_8 r_1 h r_6 chunk_9 h r_4 h'
trial_data[4]['min_program'] = p_4

In [193]:
# find shortest program

for trial_num in range(6,13):
    
    trial_programs = []
    for entry in g[0][trial_num-1].entries:
        trial_programs.append(parse(str(entry.program), base_dsl_only=False))

    ps = {p: len(p.split(' ')) for p in trial_programs}
    trial_programs_sorted = {k: v for k, v in sorted(ps.items(), key=lambda item: item[1])}
    
    trial_data.append(
    {
        'trial_num': trial_num,
        'towers': trial_seq[trial_num-1],
        'dsl_lambda': [str(p) for p in trial_grammars[trial_tasks[trial_num-1]].primitives],
        'chunks': list(map(lambda s: parse(str(s)), trial_grammars[trial_tasks[trial_num-1]].primitives[16:])),
        'dsl': base_dsl + list(map(lambda s: parse(str(s)), trial_grammars[trial_tasks[trial_num-1]].primitives[16:])),
#         'trial_programs': trial_programs_sorted,
        'min_program': list(trial_programs_sorted.keys())[0]
    })
    


In [194]:
def get_partially_chunked_programs(trial_datum):
#     chunks = trial_datum['chunks']
#     min_prog = trial_datum['min_program']
    chunk_lambdas = trial_datum['dsl_lambda'][16:]
    chunk_names = trial_datum['dsl'][28:]
    
    progs = [trial_datum['min_program']]

    for prog in progs:
        for i, chunk_name in enumerate(chunk_names):
            new_prog = prog.replace(chunk_name, parse(chunk_lambdas[i], base_dsl_only=True))
            if not(new_prog in progs):
                progs.append(new_prog)
    
    progs_with_length = {p: len(p.split(' ')) for p in progs}
    return progs_with_length

In [195]:
# find programs with abstractions replaced with base dsl only

for i, trial_datum in enumerate(trial_data):
    trial_datum['programs_with_length'] = get_partially_chunked_programs(trial_datum)

In [196]:
trial_data

[{'trial_num': 1,
  'towers': 'PiC',
  'dsl_lambda': ['2x1',
   '1x2',
   '1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   'left',
   'right'],
  'chunks': [],
  'dsl': ['h',
   'v',
   'l_0',
   'l_1',
   'l_2',
   'l_3',
   'l_4',
   'l_5',
   'l_6',
   'l_7',
   'l_8',
   'l_9',
   'l_10',
   'l_11',
   'l_12',
   'r_0',
   'r_1',
   'r_2',
   'r_3',
   'r_4',
   'r_5',
   'r_6',
   'r_7',
   'r_8',
   'r_9',
   'r_10',
   'r_11',
   'r_12'],
  'min_program': 'v r_6 v l_5 h r_4 h r_7 h l_1 v v r_1 h',
  'programs_with_length': {'v r_6 v l_5 h r_4 h r_7 h l_1 v v r_1 h': 14}},
 {'trial_num': 2,
  'towers': 'LC',
  'dsl_lambda': ['2x1',
   '1x2',
   '1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   'left',
   'right'],
  'chunks': [],
  'dsl': ['h',
   'v',
   'l_0',
   'l_1',
   'l_2',
   'l_3',
   'l_4',
   'l_5',
   'l_6',
   'l_7',
   'l_8',
   'l_9',
   'l_10',
   'l_11',
   'l_12',

In [197]:
# import json
# with open("ca_synthesis_output_manual_dechunked.json", "w") as write_file:
#     json.dump(trial_data, write_file)

In [13]:
# with open('./results/2/enumeration/enumeration_50000.p', "rb") as input_file:
#     h = pickle.load(input_file)